This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program. If not, see <https://www.gnu.org/licenses/>.

## Read Dataset <a class="anchor" name="part_00"></a>

In [2]:
from google.colab import drive
import os
from tensorflow.keras import layers, Sequential, Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory


import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize, Normalize, ToTensor, Compose, RandomVerticalFlip, RandomHorizontalFlip, RandomPerspective, RandomInvert, RandomAutocontrast

drive.mount('/content/drive')
dataset_path = '/content/drive/MyDrive/Dataset'
files = os.listdir(dataset_path)
print(files)

Mounted at /content/drive
['val', 'test', 'train', 'model_history', 'model_best.pth', 'annotated_images']


In [ ]:
def get_dataset_split_pytorch(directory, image_size=(256,256), batch_size=32):
  train_transforms = Compose([Resize(image_size), RandomVerticalFlip(), RandomHorizontalFlip(), RandomPerspective(), RandomInvert(), RandomAutocontrast(), ToTensor(), Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
  test_transforms = Compose([Resize(image_size), ToTensor(), Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

  train_data  = ImageFolder(root=f'{dataset_path}/train', transform=train_transforms)
  val_data    = ImageFolder(root=f'{dataset_path}/val', transform=test_transforms)
  test_data   = ImageFolder(root=f'{dataset_path}/test', transform=test_transforms)

  train_shape, val_shape, test_shape = len(train_data), len(val_data), len(test_data)

  train_data = DataLoader(train_data, batch_size=batch_size, shuffle=True)
  val_data = DataLoader(val_data, batch_size=batch_size, shuffle=True)
  test_data = DataLoader(test_data, batch_size=batch_size, shuffle=True)

  return train_data, val_data, test_data, train_shape, val_shape, test_shape

In [ ]:
data_augmentation = Sequential([layers.RandomFlip("horizontal_and_vertical"), layers.RandomTranslation(0.3,0.3), layers.RandomRotation(0.4), layers.RandomZoom(0.4), layers.Rescaling(1./255)])

In [ ]:
def plot_history(history, title='model', save_data=True, flag=True):
  fig, ax = plt.subplots(1, 3, figsize=(15, 5))

  ax[0].plot(history['acc'], label='train')
  ax[0].plot(history['val_acc'], label='val')
  ax[0].set_title('Model Accuracy'); ax[0].set_ylabel('Accuracy'); ax[0].set_xlabel('Epoch');
  ax[0].legend(); ax[0].grid();

  ax[1].plot(history['loss'], label='train')
  ax[1].plot(history['val_loss'], label='val')
  ax[1].set_title('Model Loss'); ax[1].set_ylabel('Loss'); ax[1].set_xlabel('Epoch');
  ax[1].legend(); ax[1].grid();

  f1_label = 'f1_m' if flag == True else 'get_f1'

  ax[2].plot(history[f1_label], label='train')
  ax[2].plot(history['val_'+f1_label], label='val')
  ax[2].set_title('Model F1 Score'); ax[2].set_ylabel('F1'); ax[2].set_xlabel('Epoch');
  ax[2].legend(); ax[2].grid();
  plt.savefig(f'{title}.png')
  plt.show()

  if save_data: np.save(f'{title}.npy', history)

In [ ]:
from keras import backend as K

def get_recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def get_precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def get_f1(y_true, y_pred):
    precision = get_precision(y_true, y_pred)
    recall = get_recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## Transfer Learning (ViT) <a class="anchor" name="part_04"></a>

In [ ]:
!pip install timm -q

In [ ]:
import torch
import timm
import torch.nn as nn

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device -- {device}')

Device -- cuda:0


In [ ]:
vit_available = timm.list_models('*vit*', pretrained=True) ## ALL ViT MODELS AVAILABLE IN PYTORCH

In [ ]:
train_data_pytorch, val_data_pytorch, test_data_pytorch, train_shape, val_shape, test_shape = get_dataset_split_pytorch(dataset_path)

### Model Structure

In [ ]:
from timm.loss import LabelSmoothingCrossEntropy
import torch.optim as optim

model = timm.create_model('vit_base_resnet50_224_in21k', pretrained=True, num_classes=2, img_size=(256,256))
model.to(device)

criterion = LabelSmoothingCrossEntropy(smoothing=0.1).cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name vit_base_resnet50_224_in21k to current vit_base_r50_s16_224.orig_in21k.
  model = create_fn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/392M [00:00<?, ?B/s]

### Train

In [ ]:

def train_model(model, train_data, val_data, n_epochs=15):
    train_losses = []
    val_losses = []
    avg_train_losses = []
    avg_val_losses = []
    avg_train_acc = []
    avg_val_acc = []

    for epoch in range(1, n_epochs + 1):
        model.train()
        correct = 0
        total = 0
        for data, target in train_data:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

            # Accuracy calculation
            _, predicted = torch.max(output, 1)
            correct += (predicted == target).sum().item()
            total += target.size(0)

        avg_train_acc.append(100 * correct / total)
        avg_train_losses.append(np.mean(train_losses))
        train_losses = []

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in val_data:
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)
                val_losses.append(loss.item())

                # Accuracy calculation
                _, predicted = torch.max(output, 1)
                correct += (predicted == target).sum().item()
                total += target.size(0)

        avg_val_acc.append(100 * correct / total)
        avg_val_losses.append(np.mean(val_losses))
        val_losses = []

        print(f'Epoch {epoch}/{n_epochs} - Train Acc: {avg_train_acc[-1]:.2f}% - Val Acc: {avg_val_acc[-1]:.2f}%')

    return model, avg_train_losses, avg_val_losses, avg_train_acc, avg_val_acc

# Ensure you have the correct train_data and val_data from the dataset
train_data, val_data, test_data, train_shape, val_shape, test_shape = get_dataset_split_pytorch(dataset_path)


In [ ]:


model, avg_train_losses, avg_val_losses, avg_train_acc, avg_val_acc = train_model(model, train_data, val_data, n_epochs=15)


Epoch 1/15 - Train Acc: 49.75% - Val Acc: 48.37%
Epoch 2/15 - Train Acc: 59.90% - Val Acc: 57.86%
Epoch 3/15 - Train Acc: 58.77% - Val Acc: 67.36%
Epoch 4/15 - Train Acc: 62.37% - Val Acc: 63.80%
Epoch 5/15 - Train Acc: 66.24% - Val Acc: 70.62%
Epoch 6/15 - Train Acc: 73.78% - Val Acc: 71.51%
Epoch 7/15 - Train Acc: 77.80% - Val Acc: 66.77%
Epoch 8/15 - Train Acc: 76.89% - Val Acc: 66.17%
Epoch 9/15 - Train Acc: 80.48% - Val Acc: 70.33%
Epoch 10/15 - Train Acc: 85.20% - Val Acc: 73.89%
Epoch 11/15 - Train Acc: 85.69% - Val Acc: 73.00%
Epoch 12/15 - Train Acc: 87.81% - Val Acc: 73.29%
Epoch 13/15 - Train Acc: 86.75% - Val Acc: 73.59%
Epoch 14/15 - Train Acc: 87.32% - Val Acc: 68.55%
Epoch 15/15 - Train Acc: 90.84% - Val Acc: 74.78%


In [ ]:


import os
import torch

# Define the path where the model will be saved
save_path = f'{dataset_path}/model_history/ViT'
model_filename = 'ViT.pth'

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Save the model
torch.save(model, os.path.join(save_path, model_filename))
print(f"Model saved at {os.path.join(save_path, model_filename)}")


Model saved at /content/drive/MyDrive/Dataset/model_history/ViT/ViT.pth


In [ ]:

model.eval()

correct = 0
total = 0

with torch.no_grad():  # No gradient computation needed during inference
    for data, target in test_data:  # test_data should be in batches
        data, target = data.to(device), target.to(device)

        # Forward pass
        output = model(data)

        # Get predictions
        _, predicted = torch.max(output, 1)

        # Compare predictions to true labels
        correct += (predicted == target).sum().item()
        total += target.size(0)

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Accuracy: 77.10%


In [ ]:

from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image
import torch

def preprocess_image(image_path):
    # Define preprocessing transformations
    transform = Compose([
        Resize((256, 256)),  # Ensure the input size matches the model
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Standard ImageNet normalization
    ])

    # Open and preprocess the image
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    return img_tensor

# Map the model to CPU if CUDA is unavailable
model_path = '/content/drive/MyDrive/Dataset/model_history/ViT/ViT.pth'
vit_model = torch.load(model_path, map_location=torch.device('cpu'))
vit_model.eval()  # Set the model to evaluation mode

# Move the model to the appropriate device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model.to(device)

def predict_image(model, image_path):
    # Preprocess the image
    img_tensor = preprocess_image(image_path).to(device)

    # Pass the image through the model
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted_class = torch.max(output, 1)  # Get the predicted class index

    # Define class labels
    class_labels = {0: "Fake", 1: "Real"}

    return class_labels[predicted_class.item()]

# Path to the image you want to classify
# image_path = f'{dataset_path}/test/real/real_00915.jpg'  # Update with your image path
image_path = '/content/drive/MyDrive/11.jpg'

# Get the prediction
prediction = predict_image(vit_model, image_path)
print(f"The image is classified as: {prediction}")


<ipython-input-8-715294b67ace>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vit_model = torch.load(model_path, map_location=torch.device('cpu'))


The image is classified as: Fake


In [ ]:


dataset_path="/content/drive/MyDrive/Dataset"



import os
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from PIL import Image, ImageDraw, ImageFont
import torch

# Preprocessing for input images
def preprocess_image(image_path):
    transform = Compose([
        Resize((256, 256)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = Image.open(image_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension
    return img_tensor

# Function to predict image class
def predict_image(model, image_path, device):
    img_tensor = preprocess_image(image_path).to(device)
    with torch.no_grad():
        output = model(img_tensor)
        probabilities = torch.softmax(output, dim=1)
        confidence, predicted_class = torch.max(probabilities, 1)
    class_labels = {0: "Fake", 1: "Real"}
    return class_labels[predicted_class.item()], confidence.item()

# Function to annotate and save the image with the prediction
def annotate_image(image_path, prediction, output_path):
    img = Image.open(image_path).convert('RGB')
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()  # Default font; customize if necessary

    # Draw the prediction text on the image
    text = f"Prediction: {prediction}"
    text_position = (10, 10)  # Position to draw the text
    text_color = (255, 0, 0)  # Red color
    draw.text(text_position, text, fill=text_color, font=font)

    # Save the annotated image
    img.save(output_path)

# Directory paths
test_dir = f'{dataset_path}/test'
output_dir = f'{dataset_path}/annotated_images'

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load the trained ViT model
model_path = '/content/drive/MyDrive/Dataset/model_history/ViT/ViT.pth'
vit_model = torch.load(model_path, map_location=torch.device('cpu'))  # Load on CPU if CUDA not available
vit_model.eval()

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model.to(device)

# Process 100 images (50 from real and 50 from fake folders)
image_count = 0
for class_dir in ['real', 'fake']:
    class_path = os.path.join(test_dir, class_dir)
    for img_name in os.listdir(class_path):
        if image_count >= 100:  # Stop after processing 100 images
            break
        image_path = os.path.join(class_path, img_name)
        prediction, confidence = predict_image(vit_model, image_path, device)
        print(f"Image: {img_name} | Prediction: {prediction} | Confidence: {confidence:.2f}")

        # Annotate and save the image
        output_path = os.path.join(output_dir, f"annotated_{img_name}")
        annotate_image(image_path, f"{prediction} ({confidence:.2f})", output_path)
        image_count += 1
    if image_count >= 100:
        break

print(f"Annotated images saved in: {output_dir}")



<ipython-input-26-1087aa680689>:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vit_model = torch.load(model_path, map_location=torch.device('cpu'))  # Load on CPU if CUDA

Image: real_00915.jpg | Prediction: Real | Confidence: 0.53
Image: real_00916.jpg | Prediction: Fake | Confidence: 0.77
Image: real_00913.jpg | Prediction: Real | Confidence: 0.92
Image: real_00914.jpg | Prediction: Real | Confidence: 0.84
Image: real_00922.jpg | Prediction: Real | Confidence: 0.79
Image: real_00918.jpg | Prediction: Fake | Confidence: 0.54
Image: real_00921.jpg | Prediction: Real | Confidence: 0.93
Image: real_00924.jpg | Prediction: Real | Confidence: 0.89
Image: real_00917.jpg | Prediction: Fake | Confidence: 0.73
Image: real_00919.jpg | Prediction: Fake | Confidence: 0.78
Image: real_00923.jpg | Prediction: Real | Confidence: 0.54
Image: real_00920.jpg | Prediction: Fake | Confidence: 0.92
Image: real_00925.jpg | Prediction: Fake | Confidence: 0.57
Image: real_00932.jpg | Prediction: Real | Confidence: 0.72
Image: real_00928.jpg | Prediction: Real | Confidence: 0.58
Image: real_00930.jpg | Prediction: Real | Confidence: 0.94
Image: real_00933.jpg | Prediction: Real